In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os
local_dir = 's18-transformer-speeding-up-strategy'
repo_url = 'https://github.com/aakashvardhan/s18-transformer-speeding-up-strategy.git'

# Check if the local directory already exists
if not os.path.exists(local_dir):
    # Clone the repository because it does not exist
    !git clone --quiet {repo_url}
else:
    # Change directory to the local repository
    %cd {local_dir}
    # Pull the latest changes because the repository already exists
    !git pull

In [3]:
import sys
sys.path.append('/content/s18-transformer-speeding-up-strategy')

In [4]:
!pip install -q -r /content/s18-transformer-speeding-up-strategy/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.2/812.2 kB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 74.2 MB/s eta 0:00:00


In [5]:
%cd /content/s18-transformer-speeding-up-strategy

/content/s18-transformer-speeding-up-strategy


In [6]:
import os
import warnings
import random

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from torchmetrics.text import BLEUScore, CharErrorRate, WordErrorRate

import lightning as L
from lightning.pytorch.callbacks import (
    EarlyStopping,
    LearningRateMonitor,
    ModelCheckpoint,
    TQDMProgressBar,
)
from lightning.pytorch.loggers import TensorBoardLogger

from config_file import get_config, get_weights_file_path
from dataset import LiTDataModule
from utils import get_model, greedy_decode

In [7]:
cfg = get_config()
# cfg['batch_size'] = 24
cfg['num_epochs'] = 18

INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42


Directory 'weights' created!
Seed set to 42...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/32332 [00:00<?, ? examples/s]

INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


...

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Max length of the source sentence : 43
Max length of the source target : 38


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


--------------------------------------------------------------------------------
    SOURCE: 'I don't understand...' said he.
    TARGET: — Non capisco — disse lui.
 PREDICTED: lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasciava lasc

KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
  File "/content/s18-transformer-speeding-up-strategy/dataset.py", line 176, in dynamic_padding_collate_fn
    return dynamic_collate_fn(batch)
  File "/content/s18-transformer-speeding-up-strategy/utils.py", line 76, in dynamic_collate_fn
    encoder_input_max = max(x["enc_token_len"] for x in batch)
  File "/content/s18-transformer-speeding-up-strategy/utils.py", line 76, in <genexpr>
    encoder_input_max = max(x["enc_token_len"] for x in batch)
KeyError: 'enc_token_len'


In [ ]:
# Define the directory name
directory_name = "weights"

# Create the directory if it does not exist
if not os.path.exists(directory_name):
  os.makedirs(directory_name)
  print(f"Directory '{directory_name}' created!")
else:
  print(f"Directory '{directory_name}' already exists.")

In [ ]:
# Clear CUDA cache and set seed
torch.cuda.empty_cache()
    L.seed_everything(42, workers=True)
    print("Seed set to 42...")

In [ ]:
# Initialize the data module
datamodule = LiTDataModule(cfg)
datamodule.setup()
print("DataModule initialized...")
tokenizer_src, tokenizer_tgt = datamodule.tokenizer_src, datamodule.tokenizer_tgt
# Initialize TensorBoard logger
tb_logger = TensorBoardLogger(
    save_dir=os.getcwd(), version=1, name="lightning_logs"
)

In [ ]:
# Initialize the trainer
trainer = L.Trainer(
            precision=cfg["precision"],
            max_epochs=cfg["num_epochs"],
            logger=tb_logger,
            accelerator=cfg["accelerator"],
            devices="auto",
            default_root_dir=cfg["model_folder"],
            callbacks=[
                ModelCheckpoint(
                    dirpath=cfg["model_folder"],
                    save_top_k=3,
                    monitor="train_loss_step",
                    mode="min",
                    filename="model-{epoch:02d}-{train_loss:.4f}",
                    save_last=True,
                ),
                LearningRateMonitor(logging_interval="step", log_momentum=True),
                EarlyStopping(
                    monitor="train_loss_step", mode="min", stopping_threshold=1.6
                ),
                TQDMProgressBar(refresh_rate=10),
            ],
            gradient_clip_val=0.5,
            num_sanity_val_steps=5,
            enable_progress_bar=True,
            check_val_every_n_epoch=1,
            limit_val_batches=2)

In [ ]:
# Initialize the model
model = LTModel(cfg, tokenizer_src=tokenizer_src, tokenizer_tgt=tokenizer_tgt)

In [ ]:
# Learning rate finder
tuner = L.pytorch.tuner.Tuner(trainer)
lr_finder = tuner.lr_find(
    model, datamodule=datamodule, num_training=trainer.max_epochs
)
print(lr_finder)

# Initialize suggested_lr with a default value
suggested_lr = cfg["one_cycle_best_lr"]

if lr_finder:
  fig = lr_finder.plot(suggest=True)
  fig.show()
  suggested_lr = lr_finder.suggestion()
  print(f"Suggested learning rate: {suggested_lr}")
else:
  print("Learning rate finding did not complete successfully.")

# Set the best learning rate
model.one_cycle_best_lr = suggested_lr

In [ ]:
trainer.fit(model=model, datamodule=datamodule)

In [ ]:
# Validate the model
trainer.validate(model=model, datamodule=datamodule)
print("Model Evaluation Done...")

In [ ]:
# Save the model
torch.save(model.state_dict(), "saved_resnet18_model.pth")
print("Model saved...")

In [ ]:
# start tensorboard
%load_ext tensorboard
%tensorboard --logdir /content/s18-transformer-speeding-up-strategy/lightning_logs